In [1]:
from pathlib import Path
import shutil

scratchpad_dir = Path("../scratchpad")

# Subdirectories to clean
subdirs = ["plots", "data", "notes"]

cleaned = []
for subdir in subdirs:
    folder = scratchpad_dir / subdir
    if folder.exists():
        # Remove all files in the folder
        for item in folder.iterdir():
            if item.is_file():
                item.unlink()
                cleaned.append(str(item))
            elif item.is_dir():
                shutil.rmtree(item)
                cleaned.append(str(item) + "/")
    else:
        # Create the folder if it doesn't exist
        folder.mkdir(parents=True, exist_ok=True)

if cleaned:
    print(f"🧹 Cleaned {len(cleaned)} items from scratchpad:")
    for item in cleaned:
        print(f"   - {item}")
else:
    print("✨ Scratchpad folders are already empty")

print(f"\n📁 Empty folders preserved:")
for subdir in subdirs:
    print(f"   - scratchpad/{subdir}/")

🧹 Cleaned 5 items from scratchpad:
   - ../scratchpad/plots/tsla_close_last5days.png
   - ../scratchpad/plots/tech_corr_heatmap.png
   - ../scratchpad/data/fed_event_bonds.csv
   - ../scratchpad/data/fed_event_equities.csv
   - ../scratchpad/data/fed_event_commodities.csv

📁 Empty folders preserved:
   - scratchpad/plots/
   - scratchpad/data/
   - scratchpad/notes/


# Analysis Sub-Agent Test Notebook

This notebook tests the analysis sub-agent, which is responsible for:
- Data analysis and processing
- Creating visualizations (saved to deep-agent/scratchpad/plots)
- Statistical analysis and trend identification
- Supporting the main agent's Markets Research & Portfolio Risk Orchestration goals

The main agent uses this sub-agent to:
- Analyze equity and factor data
- Generate price reaction analysis
- Create correlation, beta, and sector aggregation visualizations
- Execute any task requiring code execution, charts, or numerical summaries

## Setup

In [2]:
import sys
import importlib

sys.path.insert(0, '..')  # Add parent directory to path

# Import and reload to pick up any changes to the agent
from agents import analysis_agent
importlib.reload(analysis_agent)
from agents.analysis_agent import analysis_agent_graph

from langgraph.types import Command
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Analysis agent loaded successfully (with latest changes)")

✅ Analysis agent loaded successfully (with latest changes)


## Helper Function to Test the Agent

In [3]:
def test_analysis_agent(user_message: str, thread_id: str = "test-thread"):
    """
    Test the analysis agent with a user message.
    
    Args:
        user_message: The task/question for the analysis agent
        thread_id: Unique thread ID for this conversation
    
    Returns:
        The agent's final response message
    """
    print("="*80)
    print(f"USER MESSAGE:\n{user_message}")
    print("="*80)
    print("\nProcessing...\n")
    
    # Invoke the agent with a fresh conversation (NOT resuming)
    result = analysis_agent_graph.invoke(
        {"messages": [("user", user_message)]},
        config={"configurable": {"thread_id": thread_id}}
    )
    
    # Get the final AI message
    final_message = result["messages"][-1].content
    
    print("="*80)
    print("ANALYSIS AGENT RESPONSE:")
    print("="*80)
    print(final_message)
    print("\n" + "="*80)
    print("✅ Task completed. Check deep-agent/scratchpad/plots/ for any generated visualizations.")
    print("="*80 + "\n")
    
    return final_message

---
# Example 1: Basic Price Movement Visualization (Simple)

**Context**: A trader wants to quickly visualize recent price movements for a single stock.

**Sub-agent role**: Create a simple time-series visualization showing price trends.

In [4]:
# Example 1: Simple price movement analysis
example_1_message = """I need you to create a simple price movement chart for analysis.

Here's the price data for TSLA over the last 5 trading days:

Date,Close,Volume
2025-12-15,385.50,125000000
2025-12-16,392.30,138000000
2025-12-17,388.75,115000000
2025-12-18,395.20,142000000
2025-12-19,401.85,156000000

Please create a clean line chart showing the closing prices over time. 
Save it to the outputs directory so I can include it in my report.
"""

# Use a fresh thread ID to avoid any cached responses
import time
response_1 = test_analysis_agent(example_1_message, thread_id=f"example-1-{int(time.time())}")

USER MESSAGE:
I need you to create a simple price movement chart for analysis.

Here's the price data for TSLA over the last 5 trading days:

Date,Close,Volume
2025-12-15,385.50,125000000
2025-12-16,392.30,138000000
2025-12-17,388.75,115000000
2025-12-18,395.20,142000000
2025-12-19,401.85,156000000

Please create a clean line chart showing the closing prices over time. 
Save it to the outputs directory so I can include it in my report.


Processing...

ANALYSIS AGENT RESPONSE:
1. Key findings
- Created a clean line chart of TSLA closing prices over the last 5 trading days.
- Prices show an overall upward trend from 385.50 to 401.85 USD.
- There is a small dip on 2025-12-17 before resuming the upward move.

2. Visualization created
- TSLA closing price line chart:  
  `scratchpad/plots/tsla_close_last5days.png`

3. Confidence level
- High – data was parsed directly from your provided CSV and plotted without issues.

4. Caveats or limitations
- Only 5 data points are included; this is su

---
# Example 2: Sector Correlation Analysis (Medium)

**Context**: A portfolio manager wants to understand how different tech stocks moved together during a recent market event.

**Sub-agent role**: Calculate correlations between multiple stocks and create a correlation heatmap to identify risk concentrations.

In [5]:
# Example 2: Sector correlation analysis
example_2_message = """Analyze the correlation between major tech stocks during the last 10 trading days.

Here's the daily return data (%):

Date,AAPL,MSFT,GOOGL,META,NVDA
2025-12-09,0.5,0.3,0.8,1.2,2.1
2025-12-10,-0.8,-0.5,-1.1,-1.3,-2.5
2025-12-11,1.2,0.9,1.5,1.8,3.2
2025-12-12,-0.3,-0.2,-0.4,-0.6,-0.9
2025-12-13,0.9,0.7,1.1,1.4,2.3
2025-12-16,-1.5,-1.2,-1.8,-2.1,-3.4
2025-12-17,1.8,1.4,2.2,2.5,4.1
2025-12-18,0.4,0.3,0.5,0.7,1.1
2025-12-19,-0.6,-0.4,-0.8,-1.0,-1.6
2025-12-20,1.1,0.8,1.3,1.6,2.7

Please:
1. Calculate the correlation matrix between these stocks
2. Create a heatmap visualization showing the correlations
3. Identify which stocks are most correlated (potential concentration risk)
4. Save the visualization for inclusion in a risk report
"""

response_2 = test_analysis_agent(example_2_message, thread_id="example-2")

USER MESSAGE:
Analyze the correlation between major tech stocks during the last 10 trading days.

Here's the daily return data (%):

Date,AAPL,MSFT,GOOGL,META,NVDA
2025-12-09,0.5,0.3,0.8,1.2,2.1
2025-12-10,-0.8,-0.5,-1.1,-1.3,-2.5
2025-12-11,1.2,0.9,1.5,1.8,3.2
2025-12-12,-0.3,-0.2,-0.4,-0.6,-0.9
2025-12-13,0.9,0.7,1.1,1.4,2.3
2025-12-16,-1.5,-1.2,-1.8,-2.1,-3.4
2025-12-17,1.8,1.4,2.2,2.5,4.1
2025-12-18,0.4,0.3,0.5,0.7,1.1
2025-12-19,-0.6,-0.4,-0.8,-1.0,-1.6
2025-12-20,1.1,0.8,1.3,1.6,2.7

Please:
1. Calculate the correlation matrix between these stocks
2. Create a heatmap visualization showing the correlations
3. Identify which stocks are most correlated (potential concentration risk)
4. Save the visualization for inclusion in a risk report


Processing...

ANALYSIS AGENT RESPONSE:
1. **Correlation matrix (daily returns, last 10 trading days)**

(all values are Pearson correlations)

```
        AAPL   MSFT  GOOGL   META   NVDA
AAPL   1.000  0.999  0.999  0.999  0.997
MSFT   0.999  1.

---
# Example 3: Multi-Asset Event Impact Analysis (Complex)

**Context**: After a major Fed announcement, a risk manager needs to understand the cross-asset impact on their portfolio, including equities, bonds, and commodities.

**Sub-agent role**: Perform comprehensive analysis including:
- Price reaction analysis across multiple asset classes
- Volatility spike detection
- Statistical significance testing
- Multiple coordinated visualizations
- Portfolio-level impact assessment

In [6]:
# Example 3: Complex multi-asset event impact analysis
# First, write the data to CSV files in scratchpad/data

import pandas as pd
from pathlib import Path

# Create the data directory
data_dir = Path("../scratchpad/data")
data_dir.mkdir(parents=True, exist_ok=True)

# Equity indices data
equity_data = """Time,SPY,QQQ,IWM
13:00,0.0,0.0,0.0
13:30,0.1,0.2,0.0
14:00,0.2,0.3,0.1
14:30,1.5,2.1,1.2
15:00,1.8,2.5,1.4
15:30,1.6,2.3,1.3
16:00,1.5,2.2,1.2"""

# Bond yields data
bonds_data = """Time,UST_2Y,UST_10Y,UST_30Y
13:00,0,0,0
13:30,1,0,0
14:00,2,1,1
14:30,-8,-12,-10
15:00,-10,-15,-12
15:30,-9,-14,-11
16:00,-8,-13,-11"""

# Commodities data
commodities_data = """Time,Gold,Oil,Dollar_Index
13:00,0.0,0.0,0.0
13:30,0.1,-0.1,0.0
14:00,0.2,-0.1,0.1
14:30,1.8,-1.5,-1.2
15:00,2.1,-1.8,-1.4
15:30,2.0,-1.7,-1.3
16:00,1.9,-1.6,-1.2"""

# Write CSV files
import io
pd.read_csv(io.StringIO(equity_data)).to_csv(data_dir / "fed_event_equities.csv", index=False)
pd.read_csv(io.StringIO(bonds_data)).to_csv(data_dir / "fed_event_bonds.csv", index=False)
pd.read_csv(io.StringIO(commodities_data)).to_csv(data_dir / "fed_event_commodities.csv", index=False)

print(f"✅ Data files written to {data_dir.resolve()}")
for f in data_dir.glob("*.csv"):
    print(f"   - {f.name}")

# Now pass file paths to the agent
example_3_message = """Analyze the market impact of the Fed rate decision announced on 2025-12-18 at 2:00 PM EST.

I need a comprehensive analysis across multiple asset classes. The data is stored in CSV files:

- Equity indices (Intraday % change, 30-min intervals): scratchpad/data/fed_event_equities.csv
- Bond yields (Basis points change): scratchpad/data/fed_event_bonds.csv  
- Commodities (% change): scratchpad/data/fed_event_commodities.csv

PORTFOLIO EXPOSURES (as % of total portfolio):
SPY: 35%
QQQ: 25%
IWM: 10%
UST_10Y: 20%
Gold: 5%
Oil: 5%

Please provide:
1. Multi-panel visualization showing price reactions across all asset classes with a vertical line at 14:00 (announcement time)
2. Calculate the portfolio-level impact based on the exposures provided
3. Identify which asset showed the most significant reaction (using statistical measures)
4. Calculate the realized volatility spike (comparing 30 min before vs 30 min after the announcement)
5. Create a summary table showing:
   - Asset
   - Max intraday move
   - Impact on portfolio (%)
   - Statistical significance (t-stat comparing pre/post volatility)

Save all visualizations with descriptive names. This will go into a risk committee presentation.
"""

response_3 = test_analysis_agent(example_3_message, thread_id=f"example-3-{int(time.time())}")

✅ Data files written to /Users/jacobshort/Documents/code_projects/deep-agents/ibm-langgraph-deep-agents-lt-memory-talk/deep-agent/scratchpad/data
   - fed_event_bonds.csv
   - fed_event_equities.csv
   - fed_event_commodities.csv
USER MESSAGE:
Analyze the market impact of the Fed rate decision announced on 2025-12-18 at 2:00 PM EST.

I need a comprehensive analysis across multiple asset classes. The data is stored in CSV files:

- Equity indices (Intraday % change, 30-min intervals): scratchpad/data/fed_event_equities.csv
- Bond yields (Basis points change): scratchpad/data/fed_event_bonds.csv  
- Commodities (% change): scratchpad/data/fed_event_commodities.csv

PORTFOLIO EXPOSURES (as % of total portfolio):
SPY: 35%
QQQ: 25%
IWM: 10%
UST_10Y: 20%
Gold: 5%
Oil: 5%

Please provide:
1. Multi-panel visualization showing price reactions across all asset classes with a vertical line at 14:00 (announcement time)
2. Calculate the portfolio-level impact based on the exposures provided
3. Iden

---
# Verify Output Files

Check what plots were created in the scratchpad/plots directory.

In [1]:
import os
from pathlib import Path

plots_dir = Path("../scratchpad/plots")

if plots_dir.exists():
    plot_files = list(plots_dir.glob("*.png"))
    print(f"📊 Found {len(plot_files)} plots in {plots_dir}:\n")
    for plot in sorted(plot_files, key=lambda x: x.stat().st_mtime, reverse=True):
        print(f"  - {plot.name}")
else:
    print(f"❌ Directory {plots_dir} does not exist yet")

📊 Found 4 plots in ../scratchpad/plots:

  - fed_event_portfolio_impact_bar.png
  - fed_event_multi_panel.png
  - tech_correlations_heatmap.png
  - tsla_close_last5days.png


---
# Notes

## Expected Outputs

For each example, the analysis agent should:
1. **Process the data** provided in the message
2. **Execute Python code** to perform the requested analysis
3. **Generate visualizations** saved to `/home/daytona/outputs/` (auto-downloaded to `scratchpad/plots/`)
4. **Return a response** containing:
   - Key findings (3-5 bullet points)
   - Paths to visualizations created (in `scratchpad/plots/` format)
   - Confidence level in the analysis
   - Any caveats or limitations

## Testing Different Complexity Levels

- **Example 1 (Simple)**: Tests basic visualization capability
- **Example 2 (Medium)**: Tests statistical analysis and correlation calculations
- **Example 3 (Complex)**: Tests multi-faceted analysis, statistical testing, and comprehensive reporting

## Integration with Main Agent

In production, the main agent would:
1. Gather data using the `web-research-agent`
2. Save relevant data to `scratchpad/data/`
3. Delegate to `analysis-agent` with instructions + data path
4. Receive visualization paths and insights
5. Verify findings with `credibility-agent`
6. Compile everything into a PDF report using `generate_pdf_report`

---
# Cleanup

Clear generated files from scratchpad folders (keeps the empty folder structure).